In [18]:
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
test_df = pd.read_csv('Thesis/test.csv')


In [19]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

# Define Antecedents (inputs)
cross_border = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'cross_border')
country_risk = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'country_risk')
pep_involvement = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'pep_involvement')
transaction_type = ctrl.Antecedent(np.arange(0, 3, 1), 'transaction_type')

# Define Consequent (output)
risk = ctrl.Consequent(np.arange(0, 101, 1), 'risk')

# Membership Functions for PEP Involvement
pep_involvement['no'] = fuzz.trapmf(pep_involvement.universe, [0, 0, 0.3, 0.5])
pep_involvement['yes'] = fuzz.trapmf(pep_involvement.universe, [0.5, 0.7, 1, 1])

# Membership Functions for Cross Border
cross_border['low'] = fuzz.trapmf(cross_border.universe, [0, 0, 0.3, 0.5])
cross_border['high'] = fuzz.trapmf(cross_border.universe, [0.5, 0.7, 1, 1])

# Membership Functions for Country Risk
country_risk['low'] = fuzz.trapmf(country_risk.universe, [0, 0, 0.3, 0.5])
country_risk['high'] = fuzz.trapmf(country_risk.universe, [0.5, 0.7, 1, 1])

# Membership Functions for Transaction Type
transaction_type['crypto_transfer'] = fuzz.trimf(transaction_type.universe, [0, 0, 1])
transaction_type['payment'] = fuzz.trimf(transaction_type.universe, [1, 1, 2])
transaction_type['other'] = fuzz.trimf(transaction_type.universe, [2, 2, 2])

# Membership Functions for Risk
risk['low'] = fuzz.trimf(risk.universe, [0, 0, 50])
risk['medium'] = fuzz.trimf(risk.universe, [20, 50, 80])
risk['high'] = fuzz.trimf(risk.universe, [60, 100, 100])

# Define Rules
rule1 = ctrl.Rule(transaction_type['crypto_transfer'] | transaction_type['payment'], risk['high'])
rule2 = ctrl.Rule(pep_involvement['yes'] | country_risk['high'], risk['high'])
rule3 = ctrl.Rule(cross_border['high'], risk['medium'])
rule4 = ctrl.Rule(cross_border['low'] & transaction_type['other'], risk['low'])

# Create Control System and Simulation
aml_control = ctrl.ControlSystem([rule1, rule2, rule3, rule4])
aml_sim = ctrl.ControlSystemSimulation(aml_control)

# Function to evaluate a single transaction
def evaluate_transaction(row):
    transaction_type_value = {
        'CRYPTO-TRANSFER': 0,
        'PAYMENT': 1,
        'OTHER': 2
    }.get(row['Transaction_Type'], 2)  # Default to 'OTHER'

    pep_involvement_value = 1 if row['Bene_Is_Pep'] or row['Sender_Is_Pep'] else 0

    cross_border_value = 1 if row['Sender_Country'] != row['Bene_Country'] else 0

    high_risk_countries = ['Iran', 'Syria', 'North-Korea']
    country_risk_value = 1 if row['Bene_Country'] in high_risk_countries else 0

    # Feed inputs to the fuzzy control system
    aml_sim.input['transaction_type'] = transaction_type_value
    aml_sim.input['pep_involvement'] = pep_involvement_value / 10  # Normalize for membership functions
    aml_sim.input['cross_border'] = cross_border_value / 10  # Normalize for membership functions
    aml_sim.input['country_risk'] = country_risk_value / 10  # Normalize for membership functions

    # Compute the output risk
    aml_sim.compute()
    risk_score = aml_sim.output['risk']

    # Determine the label based on the risk score
    label = 1 if risk_score >= 60 else 0
    return label

test_df['money_laundering_label'] = test_df.apply(evaluate_transaction, axis=1)

# Calculate metrics
test_f1 = f1_score(test_df['Label'], test_df['money_laundering_label'])
test_precision = precision_score(test_df['Label'], test_df['money_laundering_label'])
test_recall = recall_score(test_df['Label'], test_df['money_laundering_label'])

# Generate confusion matrix
test_conf_matrix = confusion_matrix(test_df['Label'], test_df['money_laundering_label'])

# Print the metrics
print("Test set metrics:")
print(f"    F1 Score: {test_f1:.4f}")
print(f"    Precision: {test_precision:.4f}")
print(f"    Recall: {test_recall:.4f}")
print("    Confusion Matrix:")
print(test_conf_matrix)


Test set metrics:
    F1 Score: 0.4747
    Precision: 0.4938
    Recall: 0.4570
    Confusion Matrix:
[[158662  40359]
 [ 46787  39378]]
